In [1]:
import numpy as np, scipy, pandas as pd, pyDbs
from scipy import stats
rng = np.random.default_rng(seed = 105)
stats.truncnorm.random_state = rng
from symMaps.__init__ import *
from symMaps.lpSys import _adjF
_numtypes = (int,float,np.generic)

# Documentation for `LPSys` 

The ```LPSys``` is designed for Linear Programming (LP) problems. This means stacking vectors as in `symmaps.SimpleSys`, but also methods for setting up coefficient matrices that specify linear constraints on the variables.

To remind ourselves of the task at hand, we formulate models in an "augmented form" as follows (as opposed to the "standard form" that does not feature explicit equality constraints):
$$\begin{align} 
    &\min_{x} \ c^T\cdot x \tag{1A}\\ 
    &A_{ub}\times x \leq b_{ub} \tag{1B}\\ 
    &A_{eq}\times x  = b_{eq} \tag{1C}\\ 
    &l\leq x\leq u, \tag{1D}
\end{align}$$
where: 
* $x$ is the vector of choice variables of length ($N$).
* $c, l, u$ are coefficient vectors of the same length ($N$).
* $b_{eq}, b_{ub}$ are coefficient vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

We will generally refer to the inequality constraints $l\leq x \leq u$ as *domain constraints*, whereas the constraints that combine multiple variables as *variational constraints* (1B-1C). 

The `LPSys` class helps specify (1A)-(1D) building on symbols and constrains defined over indices. This is particularly helpful if we have a relatively large/complex system with many symbols and constraints and if model constraints/variables are naturally defined over indices (i.e. some structures repeated). As with before,

## 1. The overall structure and workflow of the `LPSys` class

The `LPSys` class is initialized with the following specified structure/attributes:
* `self.db`: An instance of the `pyDbs.SimpleDB` database - a simple (key, value) database with a few custom methods. Some of the useful features in this database class is that symbols are stored as `pyDbs.Gpy_` instances that ensure a common structure building on pandas series or indices. Other useful features is that they always have a `self.name` attribute that we can reference and variables have methods for returning levels, lower, and upper bounds both as pandas objects and as numpy arrays. The package `pyDbs` includes further documentation on this.
* `self.v`: Dictionary used to establish domains of symbols in $x$ from problem (1) with key = symbol name, value = domain (specified as pd.Index) of the variable. We use `value = None` to indicate scalar symbols in $x$. 
* `self.eq`: Dictionary used to establish domains of equality constraints in (1C) with key = constraint name, value = domain (specified as pd.Index). We use `value = None` to indicate a scalar constraint.
* `self.ub`: Dictionary used to establish domains of inequality constrains in (1B) with key = constraint name, value = domain (specified as pd.Index). We use `value = None` to indicate a scalar constraint.
* `self.lp`: Dictionary with either containers for the 7 vectors/matrices required to set up the linear problem - $c$, $b_{ub}$, $b_{eq}$, $l$, $u$, $A_{ub}$, and $A_{eq}$.
    * `self.lp['c']`: `pyDbs.GpyDict` instance - a simple (key,value) database with a few custom methods (akin to `pyDbs.SimpleDB`). This collects parameters for establishing the $c$-vector (examples below). *Note*: All vectors (`self.lp['c'], self.lp['b_eq'], self.lp['b_ub'], self.lp['l'], self.lp['u']`) are handled in the same way. 
    * `self.lp['A_ub']`: `symMaps.AMDict` instance - a simple (key,value) database with custom methods for setting up matrices. Matrices are themselves initialized as `symMaps.AMatrix` instances. We elaborate on the structure of these below.*Note:* The `self.lp['A_eq']` is equivalent to `self.lp['A_ub']`.
* `self.out`: Dictionary with final objects that can be passed to `scipy.optimize.linprog` (vectors `c`, `b_eq`, `b_ub`, matrices `A_ub`, `A_eq`, and `bounds` defined by stacking `l`, `u`). This is an attribute that is updated by calling the `self.compile()` method, but the object is stored as an attribute to make it possible to avoid compiling after minor adjustments to the data.
* `self.len`: Dictionary that - after compilation - contains the length $N$ of the solution vector (in `self.len['v']`), the number of rows in the equality matrix $N_{eq}$ (in `self.lp['eq']`), and the number of rows in the inequality matrix $N_{ub}$ (in `self.lp['ub']`).
* `self.maps`: Dictionary that - after compilation - contains mappings from symbols/constraint indices to "global indices".
    * `self.maps['v']` is a dictionary with mappings from pandas indices in  `self.v` to the global index for the stacked vector $x$.
    * `self.maps['eq']` is a dictionary with mappings from pandas indices in `self.eq` to the global (row) index for the stacked matrix $A_{eq}$.
    * `self.mpas['ub']` is a dictionary with mappings from pandas indices in `self.ub` to the global (row) index for the stacked matrix $A_{ub}$.
* `self.scalarDual`: Boolean that specifies how to handle values of dual variables (shadow variables) on domain constraints when $l = u$  (i.e. the bounds are the same). If `True`, the shadow value is always assigned to the upper bound (and none to the lower bound); otherwise we use default from `scipy.optimize.linprog` algorithm.

The following elaborates on all of these bits through different examples. The notebook `docs_ModelShell.ipynb` shows how it can be used to specify power system models and goes through a specific example from A-Z. 

## 2. Example of usage

### A. Domains

We start by adding a few building blocks to use in the LP system.

In [2]:
self = LPSys()

Add some time indices:

In [3]:
t = pd.Index(range(2), name = 't')
i = pd.Index(range(10,13), name = 'i')
j = pd.Index(range(20,24), name = 'j')

Add some variables defined over these indices:

In [4]:
self.v = {'p': t,
          'y': pd.MultiIndex.from_product([t,i]),
          'z': CPI([pd.MultiIndex.from_product([t,i]), j]),
          'k': None} # scalar

Add some constraints over these incides (note that equality and inequality constraints are added in exactly the same way (only calling one `eq` and the other `ub`), so we only consider `eq` constrains for simplicity her):

In [5]:
self.eq = {'sclr': None, # scalar constraint
           'vec_t': t, # entire t index
           'vec_tx0': t[1:], # t index except initial value
           'vec_tj': self.v['z'].droplevel('i').unique()} # combination of t,j that is used in z index 

This system thus splits up the solution vector $x$ from problem (1) into four symbols. If we use `self.compileMaps()`, we can see how the four symbols are going to be stacked to the vector $x$. If we look under `self.maps['v']`, we have a dictionary that maps from the domains of the four symbols ($p$, $y$, $z$, $k$), to the ordering used in the stacked solution vector:

In [6]:
self.compileMaps() # create maps
self.maps['v']['p'] # the symbol 'p' is placed first in the stacked vector. Thus, the element x[0] from the stacked vector represents the symbol p[t = 0].

t
0    0
1    1
dtype: int64

In [7]:
self.maps['v']['y'] # the symbol 'y' is placed second in the stacked vector. The small print here shows that the element x[2] represents the symbol y[t=0, i=10]

t  i 
0  10    2
   11    3
   12    4
1  10    5
   11    6
   12    7
dtype: int64

Similar maps are established for `eq` and `ub` representing the different equality and inequality constraints in the system. For instance:

In [8]:
self.maps['eq']['vec_t'] # the constraint 'vec_t' is placed second in the stacked constraints. Thus element 1 represents the constraint vec[t = 0]. 

t
0    1
1    2
dtype: int64

### B. Add vectors (1d) 

Vector coefficients are added as ```GpyDict``` instances. These store dictionaries of ```Gpy``` symbols. We add by specifying id, name, parameter, and where id is a unique identifier, name specifies what of the four symbols in `self.v` the coefficient pertains to ($p$, $y$, $z$, or $k$), and parameter is a pandas series specifying the actual coefficients. Note that this means that the index in the parameter we pass has to be consistent with what we have declared about the variable in `self.v`.

#### *i)* Examples - $c$

**Example: Add parameter that applies to part of a symbol**

*Here, we add a component with identfier 'p[t=0]' that applies to the symbol 'p'. Specifically, we add a coefficient 1 on the parts of the index t[0:1]:*

In [9]:
self.lp['c'][('p[t=0]','p')] = pd.Series(1, index = t[0:1]) # Note: The first entry is a unique identifier, the second identifies name of Gpy

*Note the parameter we add here is added with an index that is consistent with what we have referenced for $p$ in `self.v`, thus the following should return `True`:*

In [10]:
attr, comp = 'c', 'p[t=0]'
gpyInst = self.lp[attr][comp] # the specific gpy element we are checking here
assert all(gpyInst.index.isin(self.v[gpyInst.name])), f"The component {comp} in self.lp[{attr}] is defined over an index inconsistent with declared in self.v"

**Example: Add parameter on scalar symbol 'k':**

*With a scalar, by construction, we can only apply one coefficient, and thus there is not use for splitting up the specification into several components. In general, if a component specifies the coefficients for the entire domain of the symbol, we do not need to give it a unique identifier different from the symbol name:*

In [11]:
self.lp['c']['k'] = 10 # this specifies that the element in th stacked vector x that represents the scalar 'k' is 10.

*As the scalar is not defined over any index, the only check we do here to see that we have added it properly, is to check that symbol is a `GpyScalar`:*

In [12]:
attr, comp = 'c', 'k'
gpyInst = self.lp[attr][comp] # 
if gpyInst.type == 'scalar':
    assert self.v[gpyInst.name] is None, f"The component {comp} in self.lp[{attr}] is defined over an index inconsistent with declared in self.v"

*We can also add the coefficient on scalar with a different name, for instance:*

In [13]:
self.lp['c'][('otherNameForK', 'k')] = 10 # this is equivalent with the other method, except it now has a different identifier in self.lp['c']
del self.lp['c']['otherNameForK'] # delete again, we don't need it here

**Example: Add parameter on full vector for $p$**

*This adds random uniform numbers for the full vector 'p'.*

In [14]:
self.lp['c']['p'] = pd.Series(np.random.uniform(0, 1, len(self.v['p'])), index = self.v['p'], name = 'p')

**Example: Add partial vector for $z$. This leaves the rest of the model sparse.**

In [15]:
self.lp['c'][('zPart1','z')] = pd.Series(np.random.uniform(1, 2, 10), index = self.v['z'][:10])

#### *ii)* Examples - $b_{eq}, b_{ub}$

Vectors $b_{eq},b_{ub}$ are added in the exact. For example:

In [16]:
constr = 'vec_t'
self.lp['b_eq'][constr] = pd.Series(1, index = self.eq[constr]) # add ones for entire domain of the relevant constraint

This is by construction consistent with the declared domains:

In [17]:
attr, comp = 'b_eq', 'vec_t'
gpyInst = self.lp[attr][comp] # the specific gpy element we are checking here
assert all(gpyInst.index.isin(self.eq[gpyInst.name])), f"The component {comp} in self.lp[{attr}] is defined over an index inconsistent with declared in self.eq"

#### *iii)* Examples - $l, u$

Bounds are defined in exact same way cost coefficients. For instance, lower bound on the entire vector 'p':

In [18]:
self.lp['l']['p'] = pd.Series(1, index = self.v['p']) 

This is also by construction consistent with the declared domains (as we rely on those to define the index):

In [19]:
attr, comp = 'l', 'p'
gpyInst = self.lp[attr][comp] # the specific gpy element we are checking here
assert all(gpyInst.index.isin(self.v[gpyInst.name])), f"The component {comp} in self.lp[{attr}] is defined over an index inconsistent with declared in self.v"

Another example where we add upper bound on part of the domains:

In [20]:
self.lp['u'][('zPart2','z')] = pd.Series(100, index = self.v['z'][:20])
attr, comp = 'u', 'zPart2'
gpyInst = self.lp[attr][comp] # the specific gpy element we are checking here
assert all(gpyInst.index.isin(self.v[gpyInst.name])), f"The component {comp} in self.lp[{attr}] is defined over an index inconsistent with declared in self.v"

#### *iv)* Compilation

The next stage of the compilation (after `self.compileMaps()` that we looked at earlier) is to compile parameters. When we call `self.compileParams()`, we essentially build the arguments in `self.out` that are required for solving the linear program:

In [21]:
self.compileParams()
self.out.keys()

dict_keys(['c', 'A_ub', 'b_ub', 'A_eq', 'b_eq', 'bounds'])

**Cost vector, $c$:**

In [22]:
c = self.out['c']

The default value where we have not supplied any is zero. Note that we can use the mapping system in the `LPSys` to extract relevant parts of this stacked vector by calling the symbol names, for instance:

In [23]:
self(c, 'p') # extract part of vector that relates to 'p'

array([0.09080153, 0.70747261])

We can also return this as a pandas object to match it with our indices:

In [24]:
self.get(c, 'p')

t
0    0.090802
1    0.707473
Name: p, dtype: float64

**Constants, $b_{eq}, b_{ub}$:**

The parts we have added about $b_{eq}$ can similar be accessed here (note default value is zero where we have provided none):

In [25]:
beq = self.out['b_eq']
beq

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

The `__call__ ` and `get` methods are specified for variables ($x$ vector), but not constraint domains. However, it is still relatively simple to extract parts of this vector if we want eg. the pandas representation for a specific constraint:

In [26]:
attr, constr = 'eq','vec_t'
pd.Series(beq[self.maps[attr][constr].values], index = self.maps[attr][constr])

1    1.0
2    1.0
dtype: float64

**Bounds:**

While we add lower and upper bounds separately, the `scipy.optimize.linprog` requires receiving this information as one array of `bounds`. We have only added lower and upper bounds for part of the stacked vector. Wherever we have not specified anything, the lower bound is set as $0$ and the upper at `np.nan` which is (currently) interpreted as unbounded by `scipy.optimize.linprog`:

In [27]:
self.out['bounds']

array([[  1.,  nan],
       [  1.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0., 100.],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan],
       [  0.,  nan]])

#### *v)* Validation checks

In the previous sections, we have offered methods for checking that the parameters we add to the model structure are consistent. For components added to 1d vectors, we can check this by calling the validate function. This call raises a `ValueError` if there is an inconsistency, otherwise it does nothing:

In [28]:
self.validateV('zPart2','u') # add name of the element and the attribute 

Similarly, we have simple checks for all components of a specific type:

In [29]:
self.validateAllVAttr('c') # check all 'c' components in self.lp['c']

... and all 1d components across `('c','l','u','b_eq','b_ub')`:

In [30]:
self.validateAllV()

#### *vi)* Lazy methods

In the sections above, we added components to the `self.lp[attr]` instances directly. We have, however, also added a few "lazy" methods, that automatically broadcasts to suitable domains. These methods broadcasts the values we pass to the domanis specified in `self.v, self.eq, self.ub`.

**Example: Pass scalar - broadcasts to full domain**

In [31]:
self.lazyV('newCoef', series = 0.5, symbol = 'y', attr = 'c') # broadcast 0.5 to the full domain in self.v['y'] and add to self.lp['c']
self.lp['c']['newCoef'].v # results in this

t  i 
0  10    0.5
   11    0.5
   12    0.5
1  10    0.5
   11    0.5
   12    0.5
dtype: float64

**Example: Specify parameter on part of the relevant domain - broadcast to full.**

The variable 'z' is defined over three indices `['t','i','j']`. Let's say that we specify a parameter that is constant across `['i','j']`, but may differ across $t$. We can then add this by specifying the variation in $t$:

In [32]:
zc_t = pd.Series(np.linspace(10, 100, len(t)), index = t) # some coefficients defined over t
self.lazyV('upperOnZ', series = zc_t, symbol = 'z', attr = 'u') # add upper bounds on the variable, broadcast from the variation in 'z'
self.lp['u']['upperOnZ'].v

t  i   j 
0  10  20     10.0
       21     10.0
       22     10.0
       23     10.0
   11  20     10.0
       21     10.0
       22     10.0
       23     10.0
   12  20     10.0
       21     10.0
       22     10.0
       23     10.0
1  10  20    100.0
       21    100.0
       22    100.0
       23    100.0
   11  20    100.0
       21    100.0
       22    100.0
       23    100.0
   12  20    100.0
       21    100.0
       22    100.0
       23    100.0
dtype: float64

### C. Matrices

Coefficients in matrices `A_eq, A_ub` are added using the small `AMtrix` class and stored in `AMDict`; these are small classes used to make sure that we all relevant information to use these in the `LPSys` structure. We start with quick explanation of the classes and follow up with some examples.

#### *i)* `AMDict`

```AMatrix``` instances are simple objects with 6 attributes: 
* name: (unique identifier used to navigate dictionary ```AMDict```),
* values: np.array (or similar) with coefficients to be applied.
* v: name of variable the cofficient applies to, 
* constr: name of constraint the coefficient applies to,
* vIdx: ```pd.Index``` corresponding to variable domains. If None = scalar. 
* constrIdx: ```pd.Index``` corresponding to index comains. If None = scalar constraint.

We can get a dataframe representation of the `AMatrix` with the index = constraint index and the variable index added as columns by calling the `AMatrix.to_frame` property.

The most flexible/unstructured way of specifying coefficients in a linear constraint, is to add these parts separately. Consider an equality constraint $q$ defined in `self.eq` defined over $j_q$, where $j_q$ can be `None` (scalar) or a pandas index over some domains. A general formulation of a linear constraint in our model system can be written as:
$$\begin{align}
    b[j_q] = \sum_{v}\left(\sum_{i_{v}} a[j_q,i_v] \times v[i_v]\right),
\end{align}$$
where $v$ refers to the symbols that the stacked vector $x$ contains, $b$ is the vector of constants for the specific constraint $q$, $j_q$ is the constraint index, $i_{v}$ is the varible index for the specific symbol $v$. 

One instance of the  `AMatrix` class is a sparse representation of the coefficients $a[j_q,i_{v}]$ that states how variable $v$ enters constraint $q$; the `LPSys` class automatically fills in zeros whereever we have not specified coefficients. The attributes of the `AMatrix` relates to this mathematical formulation as follows:
* `name`: Identifier (unrelated to math).
* `v`: Name of symbol $v$ (as declared in `self.v` in the `LPSys` instance).
* `constr`: Name of constraint $q$ (as declared in `self.eq` or `self.ub` in the `LPSys` instance).
* `values`: Vector of values in $a[j_q, i_v]$.
* `vIdx`: Index $i_v$ of the same length as `values` (i.e. elements in this index are not unique). Elements must be part of the declared domains in `self.v[v]` for variable $v$.
    * *Note: If the variable is a scalar, `vIdx = None` instead of a vector and does not have the same length as `self.values`.*
* `constrIdx`: Index $j_q$ of the same length as `values` (i.e. elements in this index are not unique). Elements must be part of declared domains in `self.eq[q]` or `self.ub[q]` for constraint $q$.
    * *Note: If the constraint is a scalar, `constrIdx = None` instead of a vector and does not have the same length as `self.values`.*

Consider the constraint 'vec_tj' defined over [t,j] and the variable `p` defined only over $t$. We can add a somewhat funky constraint that sums over all $t$ in $p[t]$ in *every* constraint index $t$, that is something like this 
$$\begin{align}
    \sum_{t'} a[t, j, t'] p[t'],
\end{align}$$
where $t'$ is an *alias* of the $t$ index (i.e. it contains same element, but with distinct name). Let us say $a = 1$ for all the combinations. We add this by specifying:

In [33]:
name = 'funkyConstraint'
v, constr = 'p', 'vec_tj'
fullIndex = pyDbs.cartesianProductIndex([self.eq[constr], _adjF(self.v[v], alias = {'t':'tt'})]) # all combinations 
values = np.ones(len(fullIndex)) # add ones for all combinations
constrIdx = pd.MultiIndex.from_frame(fullIndex.to_frame(index=False)[self.eq[constr].names])
vIdx = fullIndex.get_level_values('tt').rename('t')
Aex = AMatrix('funkyConstr', values = values, v = v, constr = constr, vIdx = vIdx, constrIdx = constrIdx)
Aex.to_frame.head() # inspect some of the dataframe

t  __values__
t j                
0 20  0         1.0
  20  1         1.0
  21  0         1.0
  21  1         1.0
  22  0         1.0

We can validate the structure of an `AMatrix` instance in two ways:
1. Internal consistency: The value of `values`, `vIdx`, and `constrIdx` has to be identical *except* if an index is `None` (scalar variable/constraints). This is checked by calling the property `self.validate`: This raises a `ValueError` if there is an issue and nothing if everything is in order.
2. External/system consistency: We can check for consistency between variable and constraint domains and the declared domains in the `LPSys`. This can be done by calling `self.validateA(name, attr)` from the `LPSys` instance with `name` being the name of the `AMatrix` instance and `attr` whether it is specified as an equality (`attr = eq`) or inequality constraint (`attr = ineq`). We return to this issue later.

In [34]:
Aex.validate # internal consistency - does nothing if everything is ok.

#### *ii) Add from pd.Series* - `self.initA_infer`

In most instances, conditions that are defined over some index, e.g. $t$, relate to variables that are defined over the same index $t$ in a more systematic way. When this is the case, a shortcut for adding $A$-coefficients can be to specify them as `pd.Series` over one index that contains information on both `vIdx` and `constrIdx`. Consider, for instance, the constraint `vec_t` that we defined as one that is repeated for all $t$ and the variable $y[t,i]$. We can specify a `pd.Series` defined over $(t,i)$ and let the `LPSys` infer what part of the index reflects `vIdx` and `constrIdx` as follows:

In [35]:
constr, v = 'vec_t','y' # name of symbols
name = f'{constr}{v}' # unique identifier in AMDict
series = pd.Series(10, index = self.v[v]) # create pandas series defined over [t,i] 
self.initA_infer(name, series = series, v = v, constr = constr, attr = 'eq')

The `self.initA_infer` method looks the domains for the constraint and variables and matches them with the pandas index:

In [36]:
self.lp['A_eq'][name].to_frame.head()

,t,i,__values__
t,,,
0,0,10,10
0,0,11,10
0,0,12,10
1,1,10,10
1,1,11,10


**Example: Scalar constraint, random coefficients over variable $z$:**

In [37]:
constr, v = 'sclr', 'z'
name = f'{constr}{v}' # unique identifier, doesn't have to be added with syntax 
series = pd.Series(1, index = self.v[v])
self.initA_infer(name, series = series, v = v, constr = constr, attr = 'eq')

In this case, the index represents variable domains, the constraint index is simply None:

In [38]:
self.lp['A_eq'][name].constrIdx is None

True

#### *iii) Add lagged/leaded coefficients from pd.Series* - `self.initA_lag, self.initA_lead`

In the example above where both constraint and variable are defined over $t$, we may want to specify a lagged constraint. Let $b$ denote the constant in the equation in (1C) and $a_{t,i}$ the coefficients in the $A_{eq}$ matrix with $t$ denoting rows and $i$ the rows. Say we want to add a lagged constraint in the following sense:  
$$\begin{align}
    b[t] = \sum_i a_{t-1,i} y_{t-1,i}.
\end{align}$$
The `self.initA_lag` method does this for us based on a `pd.Series`: We specify the $a_{t,i}$ coefficients in the object `series` below and state that the variable index $t$ should be lagged by 1 element:

In [39]:
constr, v = 'vec_tx0', 'y' 
name = f'{constr}{v}'
series = pd.Series(np.random.uniform(0,1,len(self.v[v])), index = self.v[v])
lag, level = 1, 't' # specification of lag-structure
self.initA_lag(name, series, v= v, constr = constr, attr = 'eq', lag = lag, level = level)

Importantly, note that we specified the $a_{t,i}$ coefficients for all $t$; in our case that is simply $t = 0,1$. However, if we try and evaluate the constraint for $t=0$, we evaluate the variable in $y_{-1,i}$, which is outside our specified domain for the index $t$. The default option here is therefore to remove $t=0$ in the constraint index.

In [40]:
self.lp['A_eq'][name].to_frame

,t,i,__values__
t,,,
1,0,10,0.606527
1,0,11,0.163885
1,0,12,0.352435


An alternative formulation is to define the `series` parameter on the correct domain to start with (i.e. only for $t=0$ in our case):

In [41]:
series_v2 = _adjF(series, t[:-1]) # series only defined for t=0

The `Lag.series` method maps the $t=0$ to $t=1$ and concludes that this is beyond the original domain, but if we add `fkeep = True` the forwarded indices are kept. Thus we call:

In [42]:
self.initA_lag(name, series_v2, v = v, constr = constr, attr = 'eq', lag = lag, level = level, fkeep = True)
self.lp['A_eq'][name].to_frame

,t,i,__values__
t,,,
1,0,10,0.606527
1,0,11,0.163885
1,0,12,0.352435


#### *iv) Add rolled index from pd.Series* - `self.initA_roll`

Another way to adjust the relation between constraint index and variable index, is to roll the index; this is similar to leading the index (inverse of lagging), but assumes that the index is circular. Thus, with index values (1,2,3) rolling this 1 step yields (3,1,2) instead. Note that this operates using the sorting of the index levels; this means that it works on non-numerical indices as well, but that the sorting becomes essential. The following essentially models the following equations:
$$\begin{align}
    \text{For t>0}: && b[t] &= \sum_i a_{t-1,i} y_{t-1,i}, \\
                    && b[0] &= \sum_i a_{T,i} y_{T,i},
\end{align}$$
where the second equation ties the last ($T$) and the initial ($0$) elements together:

In [43]:
constr, v = 'vec_t', 'y'
name = f'{constr}{v}'
self.initA_roll(name, series, v = v, constr = constr, attr = 'eq', roll = -1, level= 't') 

#### *v) Lazy methods*

In the examples outlined above, we specify the matrices using pandas series that combine variable and constraint indices in the same multiindex object. We have added a "lazy" approach that braodcasts the `pd.Series` we input to the domains of the variable and constraints. Consider again the generic constraint on the form:
$$\begin{align}
    b[j_q] = \sum_{v}\left(\sum_{i_{v}} a[j_q,i_v] \times v[i_v]\right),
\end{align}$$
where $q$ is constraint, $v$ is the variable, $j_q$ is the constraint domain, and $i_v$ is the variable domain. The "lazy" approach specifies a parameter $\alpha[l_q,k_v]$ defined over subsets of the full domains $j_q, i_v$ and broadcasts them to the full domains when initializating an `AMatrix`. 

More specifically, the `self.lazyA(name, series = None, v = None, constr = None, attr = None, constrIdx = None, vIdx = None)` method works as follows:
1. Specify the constraint type (`attr` $\in$ {`eq`, `ub`}), the name of the component added to `self.lp[f'A_{eq}']` (`name`), and names of the constraint (`constr`) and variable (`v`).
2. Specify coefficients in `series` (as pd.Series or scalar).
3. If `vIdx` is `None`, the `series` object is broadcasted to the full domain specified in `self.v[v]`. Alternatively, we can specify `vIdx` as a pandas index that is a subset of the full domains in `self.v[v]`; in this case, this will be the domain that `series` is broadcasted to.
4. If `constrIdx` is `None`, the `series` object is broadcasted to the full domain specified in `getattr(self, attr)[constr]`. Alternatively, we can specify `constrIdx` as a pandas index that is a subset of the full domains in `getattr(self, attr)[constr]`; in this case, this will be the domain that `series` is broadcasted to.

Once the `series` has been broadcasted to the full domains of the relevant variable and constraint, the `series` is added to the `self.lp` structure in the same way as `self.initA_infer`.

Note: The main method is implemented as ```self.lazyA```. Versions that lag/roll the variable indices are defined as ```self.lazyA_lag```, ```self.lazyA_roll``` (similar extension as explained earlier).

**Example: Broadcast scalar to full indices**

In [44]:
constr, v = 'vec_tj', 'z'
name = f'{constr}{v}'
attr = 'eq'
self.lazyA(name, series = np.e, v = v, constr = constr, attr = attr)
self.lp[f'A_{attr}'][name].to_frame.head()

t   i   j  __values__
t j                        
0 20  0  10  20    2.718282
  21  0  10  21    2.718282
  22  0  10  22    2.718282
  23  0  10  23    2.718282
  20  0  11  20    2.718282

#### *vi) Validation*

As for the 1d vectors, we also have automatic functions checking for the validity of the different matrix components. The `validateA(name, attr)` checks component `name` in `self.lp[f'A_{attr}']` and returns a `ValueError` is there is a problem, otherwise nothing:

In [45]:
self.validateA('vec_tjz', 'eq') 

Similarly, we can check all components in a type:

In [46]:
self.validateAllAAttr('eq')

We can check all matrices:

In [47]:
self.validateAllA()

... and we can validate the entire `self.lp` structure (including 1d objects):

In [48]:
self.validateAll()

## 3. Extract solution

```__call__``` method extracts part of np.array that represents the symbol we query:

In [49]:
x = np.random.uniform(0,1,self.len['v'])
self(x, 'p') # extract part of stacked vector that relates to symbol 'p'

array([0.21313453, 0.87753367])

```get``` method returns ```pd.Series``` (or scalar):

In [50]:
self.get(x,'p') # returns pandas series with values from the numpy array

t
0    0.213135
1    0.877534
Name: p, dtype: float64

To test the 'extract solution part', set up a small model that is feasible and solve:

In [51]:
self.lp['A_eq'].symbols

{'vec_ty': <symMaps.lpSys.AMatrix at 0x1c6130f8cb0>,
 'sclrz': <symMaps.lpSys.AMatrix at 0x1c613026850>,
 'vec_tx0y': <symMaps.lpSys.AMatrix at 0x1c612fa7bb0>,
 'vec_tjz': <symMaps.lpSys.AMatrix at 0x1c612fe7350>}

In [52]:
delitems = [k for k in (self.lp['A_eq'].symbols) if k != 'vec_ty']
[self.lp['A_eq'].__delitem__(k) for k in delitems];
self.compileMaps()
self.compileParams();
self.out['bounds'] = np.vstack([np.zeros(self.len['v']), np.full(self.len['v'], 10)]).T # this update of the bounds is deleted if we run a standard "compileParams" statement. 
sol = scipy.optimize.linprog(**self.out)

The solution is unloaded as dictionaries of pandas series elements in the following blocks:
* ```self.unloadSolX``` returns dictionary with solution for variables.
* ```self.unloadSolDualEq, self.unloadSolDualUb ``` returns dictionary with shadow values (duals) for the equality and inequality contraints, respectively.
* ```self.unloadSolDualLower, self.unloadSolDualUpper``` returns dictionary with shadow values (duals) for the lower/upper bounds on variables.

The ```self.unloadSol``` method combines all of the above in one:

In [53]:
solDict = self.unloadSol(sol)
solDict.keys() # a look at the symbols added from this stage

dict_keys(['p', 'y', 'z', 'k', 'λeq_sclr', 'λeq_vec_t', 'λeq_vec_tx0', 'λeq_vec_tj', 'λl_p', 'λl_y', 'λl_z', 'λl_k', 'λu_p', 'λu_y', 'λu_z', 'λu_k'])